# End-to-end Reusable ML Pipeline with Seldon and Kubeflow on GKE

In this example we showcase how to build re-usable components to build an ML pipeline that can be trained and deployed at scale.

We will create, train and deploy an income classifier model in Kubeflow with the following components:

![](img/completed-pipeline-deploy.jpg)

This tutorial will break down in the following sections:

1) Run all the services (Kubeflow and Seldon)

2) Test and build all our reusable pipeline steps

3) Use Kubeflow to Train the Pipeline and Deploy to Seldon

5) Test Seldon Deployed ML REST Endpoints

6) Visualise Seldon's Production ML Pipelines

# Before you start
Make sure you install the following dependencies, as they are critical for this example to work:

* Helm v2.13.1+
* A Kubernetes cluster running v1.13 or above (minkube / docker-for-windows work well if enough RAM)
* kubectl v1.14+
* ksonnet v0.13.1+
* kfctl 0.5.1 - Please use this exact version as there are major changes every few months
* Python 3.6+
* Python DEV requirements (we'll install them below)

Let's get started! 🚀🔥 We will be building the end-to-end pipeline below:

![](img/kubeflow-seldon-nlp-full.jpg)


In [1]:
!cat requirements-dev.txt

xai==0.0.5
python-dateutil==2.8.0
seldon_core==0.3.0
alibi==0.2.0
jupyter==1.0.0
dill==0.2.9
scikit-learn==0.20.1


In [3]:
!pip install -r requirements-dev.txt

### Initialise a GKE Cluster

Connect to the cluster:

In [17]:
!gcloud config set project dev-joel
!gcloud container clusters get-credentials standard-cluster-1 --zone=us-central1-a
!kubectl config current-context

Updated property [core/project].
Fetching cluster endpoint and auth data.
kubeconfig entry generated for standard-cluster-1.
gke_dev-joel_us-central1-a_standard-cluster-1


## 1) Run all the services (Kubeflow and Seldon)
Kubeflow's CLI allows us to create a project which will allow us to build the configuration we need to deploy our kubeflow and seldon clusters.

In [4]:
!kfctl init kubeflow-seldon
!ls kubeflow-seldon

app.yaml


Now we run the following commands to basically launch our Kubeflow cluster with all its components. 

It may take a while to download all the images for Kubeflow so feel free to make yourself a cup of ☕.

If you have a terminal you can see how the containers are created in real-time by running `kubectl get pods -n kubeflow -w`.

In [18]:
%%bash
cd kubeflow-seldon
kfctl generate all -V
kfctl apply all -V

time="2019-07-06T11:45:16+01:00" level=info msg="reading from /Users/Seldon/seldon-core/examples/income-kubeflow/kubeflow-seldon/app.yaml" filename="coordinator/coordinator.go:341"
time="2019-07-06T11:45:16+01:00" level=info msg="reading from /Users/Seldon/seldon-core/examples/income-kubeflow/kubeflow-seldon/app.yaml" filename="coordinator/coordinator.go:341"
time="2019-07-06T11:45:16+01:00" level=info msg="Ksonnet.Generate Name kubeflow-seldon AppDir /Users/Seldon/seldon-core/examples/income-kubeflow/kubeflow-seldon Platform " filename="ksonnet/ksonnet.go:369"
time="2019-07-06T11:45:17+01:00" level=info msg="Creating environment \"default\" with namespace \"kubeflow\", pointing to \"version:v1.12.8\" cluster at address \"https://35.202.37.142\"" filename="env/create.go:77"
time="2019-07-06T11:45:21+01:00" level=info msg="Generating ksonnet-lib data at path '/Users/Seldon/seldon-core/examples/income-kubeflow/kubeflow-seldon/ks_app/lib/ksonnet-lib/v1.12.8'" filename="lib/lib.go:148"
tim

### Now let's run Seldon 

Before installing Seldon Core, we need to install HELM

To do so, we need to creat a ClusterRoleBinding for us, a ServiceAccount and then a RoleBinding

In [19]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [20]:
!kubectl create serviceaccount tiller --namespace kube-system

serviceaccount/tiller created


In [21]:
!kubectl apply -f tiller-role-binding.yaml

clusterrolebinding.rbac.authorization.k8s.io/tiller-role-binding created


#### Once that is set-up we can install Tiller

In [22]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "stable" chart repository
Update Complete.


In [23]:
!helm init --service-account tiller

$HELM_HOME has been configured at /Users/Seldon/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation


In [24]:
# Wait until Tiller finishes
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


#### Now we can install SELDON.

We first start with the custom resource definitions (CRDs)

In [25]:
!helm install seldon-core-operator --namespace kubeflow --repo https://storage.googleapis.com/seldon-charts

NAME:   stultified-moth
E0706 11:53:48.534672   88559 portforward.go:372] error copying from remote stream to local connection: readfrom tcp4 127.0.0.1:60837->127.0.0.1:60841: write tcp4 127.0.0.1:60837->127.0.0.1:60841: write: broken pipe
LAST DEPLOYED: Sat Jul  6 11:53:46 2019
NAMESPACE: kubeflow
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  2s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  1s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         1s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     2s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.11.

Check all the Seldon Deployment is running

In [26]:
!kubectl get pod -n kubeflow | grep seldon

seldon-operator-controller-manager-0                       1/1     Running   1          13s


### Temporary fix for Argo image

At the time of writing we need to make some updates in the Argo images with the following commands below.

(This basically changes the images to the latest ones, otherwise we will get an error when we attach the volume)


In [27]:
!kubectl -n kubeflow patch deployments. workflow-controller --patch '{"spec": {"template": {"spec": {"containers": [{"name": "workflow-controller", "image": "argoproj/workflow-controller:v2.3.0-rc3"}]}}}}'
!kubectl -n kubeflow patch deployments. ml-pipeline --patch '{"spec": {"template": {"spec": {"containers": [{"name": "ml-pipeline-api-server", "image": "elikatsis/ml-pipeline-api-server:0.1.18-pick-1289"}]}}}}'
# !kubectl -n kubeflow patch configmaps workflow-controller-configmap --patch '{"data": {"config": "{ executorImage: argoproj/argoexec:v2.3.0-rc3,artifactRepository:{s3: {bucket: mlpipeline,keyPrefix: artifacts,endpoint: minio-service.kubeflow:9000,insecure: true,accessKeySecret: {name: mlpipeline-minio-artifact,key: accesskey},secretKeySecret: {name: mlpipeline-minio-artifact,key: secretkey}}}}" }}'

deployment.extensions/workflow-controller patched
deployment.extensions/ml-pipeline patched


The last command you need to run actually needs to be manual as the patch cannot change configmap contents directly

You need to run the edit commad and change the executorImage to: `argoproj/argoexec:v2.3.0-rc3`

The command should be run from a terminal:

```
kubectl edit configmaps workflow-controller-configmap -n kubeflow
```

### GKE Specific set up

To run this example in GKE, we need to configure a NFS Persistent volume. The shell script below intalls the required nfs clients via Helm.

In [47]:
!sh nfs-setup.sh

ERROR: (gcloud.beta.filestore.instances.create) INVALID_ARGUMENT: reserved ip range 10.0.0.0/29 collides with already allocated range 10.0.0.0/29.
NAME:   nfs-cp
LAST DEPLOYED: Sat Jul  6 12:33:51 2019
NAMESPACE: kubeflow
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                                  AGE
nfs-cp-nfs-client-provisioner-runner  1s

==> v1/ClusterRoleBinding
NAME                               AGE
run-nfs-cp-nfs-client-provisioner  1s

==> v1/Deployment
NAME                           READY  UP-TO-DATE  AVAILABLE  AGE
nfs-cp-nfs-client-provisioner  0/1    1           0          1s

==> v1/Pod(related)
NAME                                            READY  STATUS             RESTARTS  AGE
nfs-cp-nfs-client-provisioner-56968f8f98-t9vvf  0/1    ContainerCreating  0         1s

==> v1/Role
NAME                                          AGE
leader-locking-nfs-cp-nfs-client-provisioner  1s

==> v1/RoleBinding
NAME                                          AGE
leader-locking-nf

We also need to add a service account admin cluster role binding for our pipeline:

In [48]:
!kubectl create clusterrolebinding sa-admin --clusterrole=cluster-admin --serviceaccount=kubeflow:pipeline-runner

clusterrolebinding.rbac.authorization.k8s.io/sa-admin created


## 2) Test and build all our reusable pipeline steps

We will start by building each of the components in our ML pipeline. 

![](img/kubeflow-seldon-nlp-reusable-components.jpg)

### Let's first have a look at our income_classifier step:


In [28]:
!ls pipeline/pipeline_steps

income_classifier


Like in this step, all of the other steps can be found in the `pipeline/pipeline_steps/` folder, and all have the following structure:
* `pipeline_step.py` which exposes the functionality through a CLI 
* `Transformer.py` which transforms the data accordingly
* `requirements.txt` which states the python dependencies to run
* `build_image.sh` which uses `s2i` to build the image with one line

### Let's check out the CLI for income_classifier
The pipeline_step CLI is the entry point for the kubeflow image as it will be able to pass any relevant parameters


In [29]:
!python pipeline/pipeline_steps/income_classifier/pipeline_step.py --help

W0706 11:56:00.421277 140735804597120 deprecation_wrapper.py:119] From /Users/Seldon/miniconda3/lib/python3.7/site-packages/alibi/explainers/cem.py:19: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

Using TensorFlow backend.
Usage: pipeline_step.py [OPTIONS]

Options:
  --preprocessor-path TEXT
  --model-path TEXT
  --out-path TEXT
  --action [predict|train]
  --help                    Show this message and exit.


This is actually a very simple file, as we are using the click library to define the commands:


In [30]:
!cat pipeline/pipeline_steps/income_classifier/pipeline_step.py

import click
import numpy as np
import dill
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from joblib import dump
from alibi.datasets import adult


@click.command()
@click.option('--preprocessor-path', default="/mnt/preprocessor.model")
@click.option('--model-path', default="/mnt/income_class.model")
@click.option('--out-path', default="/mnt/clf_prediction.data")
@click.option('--action', default="predict", 
        type=click.Choice(['predict', 'train']))

def run_pipeline(
        preprocessor_path,
        model_path,
        out_path, 
        action):

    # load data
    data, labels, feature_names, category_map = adult()

    # define train and test set
    np.random.seed(0)
    data_perm = np.random.permutation(np.c_[data, labels])
    data = data_perm[:, :-1]
    labels = data_perm[:,

The Transformer is where the data munging and transformation stage comes in, which will be wrapped by the container and exposed through the Seldon Engine to ensure our pipeline can be used in production.

Seldon provides multiple different features, such as abilities to send custom metrics, pre-process / post-process data and more. In this example we will only be exposing the `predict` step.

In [31]:
!cat pipeline/pipeline_steps/income_classifier/Transformer.py


import dill
import logging

class Transformer(object):
    def __init__(self):

        with open('/mnt/income_class.model', 'rb') as model_file:
            self.clf_model = dill.load(model_file)

    def predict(self, X, feature_names):
        logging.warning(X)
        prediction = self.clf_model.predict_proba(X)
        logging.warning(prediction)
        return prediction




If you want to understand how the CLI pipeline talks to each other, have a look at the end to end test in `pipeline/pipeline_tests/`:

In [33]:
!pytest ./pipeline/pipeline_tests/. --disable-pytest-warnings

============================= test session starts ==============================
platform darwin -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0
rootdir: /Users/Seldon/seldon-core/examples/income-kubeflow
collected 1 item                                                               

pipeline/pipeline_tests/test_pipeline.py .                               [100%]

==================== 1 passed, 9 warnings in 22.16 seconds =====================


To build the image we provide a build script in each of the steps that contains the instructions:

In [34]:
!cat pipeline/pipeline_steps/income_classifier/build_image.sh

#!/bin/bash

s2i build . seldonio/seldon-core-s2i-python3:0.6 income_classifier:0.1



The only thing you need to make sure is that Seldon knows how to wrap the right model and file.

This can be achieved with the s2i/environment file. 

As you can see, here we just tell it we want it to use our `Transformer.py` file:


In [35]:
!cat pipeline/pipeline_steps/income_classifier/.s2i/environment

MODEL_NAME=Transformer
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0


Once this is defined, the only thing we need to do is to run the `build_image.sh` for all the reusable components.

Here we show the manual way to do it:

In [36]:
%%bash
# we must be in the same directory
cd pipeline/pipeline_steps/income_classifier/ && ./build_image.sh

---> Installing application source...
---> Installing dependencies ...
ERROR: Invalid requirement: '~'

You should consider upgrading via the 'pip install --upgrade pip' command.
Build failed
ERROR: An error occurred: non-zero (13) exit code from seldonio/seldon-core-s2i-python3:0.6


CalledProcessError: Command 'b'# we must be in the same directory\ncd pipeline/pipeline_steps/income_classifier/ && ./build_image.sh\n'' returned non-zero exit status 1.

### Push the Built Image

Push the Income Classifier image to the container registry of YOUR project

In [46]:
!gcloud auth configure-docker
!docker tag income_classifier:0.1 gcr.io/dev-joel/income_classifier:0.1
!docker push gcr.io/dev-joel/income_classifier:0.1

gcloud credential helpers already registered correctly.
The push refers to repository [gcr.io/dev-joel/income_classifier]

3df635dc: Preparing 
dfe92586: Preparing 
a212fd4b: Preparing 
a8287d6a: Preparing 
18efa189: Preparing 
03213383: Preparing 
86989c42: Preparing 
c34d0641: Preparing 
b148a147: Preparing 
fbfde642: Preparing 
9df8b8f8: Preparing 
c31e431b: Preparing 
127eee44: Preparing 
29a174eb: Preparing 


df635dc: Pushing  546.8MB/849.7MB14APushing   52.2MB/849.7MBPushing  130.6MB/849.7MBPushing  394.3MB/849.7MBPushing  459.6MB/849.7MB

df635dc: Pushed   858.8MB/849.7MBPushing  780.6MB/849.7MB0.1: digest: sha256:30618fd7a6d80a6ffbb5b92d69c776a605709791a864eedac7e85a5ce4a0210c size: 3483


## 3) Train our NLP Pipeline through the Kubeflow UI
We can access the Kubeflow dashboard to train our ML pipeline via http://localhost/_/pipeline-dashboard

If you can't edit this, you need to make sure that the ambassador gateway service is accessible:


In [37]:
!kubectl get svc ambassador -n kubeflow

NAME         TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
ambassador   NodePort   10.11.242.189   <none>        80:32535/TCP   33m


In my case, I need to change the kind from `NodePort` into `LoadBalancer` which can be done with the following command:


In [38]:
!kubectl patch svc ambassador --type='json' -p '[{"op":"replace","path":"/spec/type","value":"LoadBalancer"}]' -n kubeflow

service/ambassador patched


Now that I've changed it to a loadbalancer, it has allocated the external IP as my localhost so I can access it at http://localhost/_/pipeline-dashboard


In [45]:
!kubectl get svc ambassador -n kubeflow

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)        AGE
ambassador   LoadBalancer   10.11.242.189   35.225.29.37   80:32535/TCP   35m


If this was successfull, you should be able to access the dashboard
![](img/k-pipeline-dashboard.jpg)

### Define the pipeline
Now we want to generate the pipeline. For this we can use the DSL provided by kubeflow to define the actual steps required. 

The pipeline will look as follows:

![](img/kubeflow-seldon-nlp-ml-pipelines.jpg)

In [52]:
!cat train_pipeline/income_pipeline.py


import kfp.dsl as dsl
import yaml
from kubernetes import client as k8s


@dsl.pipeline(
  name='Income Classifier',
  description='A pipeline demonstrating reproducible steps for NLP'
)
def nlp_pipeline(
        preprocessor_path="/mnt/preprocessor.model",
        model_path="/mnt/income_class.model",
        out_path="/mnt/clf_prediction.data"):
    """
    Pipeline 
    """
    vop = dsl.VolumeOp(
        name='my-pvc',
        resource_name="my-pvc",
        modes=["ReadWriteMany"],
        storage_class="nfs-client",
        size="1Gi"
    )

    predict_step = dsl.ContainerOp(
        name='predictor',
        image='gcr.io/dev-joel/income_classifier:0.1',
        command="python",
        arguments=[
            "/microservice/pipeline_step.py",
            "--preprocessor-path", preprocessor_path,
            "--model-path", model_path,
            "--out-path", out_path,
            "--action", "train",
        ],
        pvolumes={"/mnt": vop.volume}
    )

    try:
        s

### Breaking down the  code
As you can see in the DSL, we have the ContainerOp - each of those is a step in the Kubeflow pipeline.

At the end we can see the `seldondeploy` step which basically deploys the trained pipeline

The definition of the SeldonDeployment graph is provided in the `deploy_pipeline/seldon_production_pipeline.yaml` file.

The seldondeployment file defines our production execution graph using the same reusable components.

In [50]:
!cat deploy_pipeline/seldon_production_pipeline.yaml

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: "seldon-deployment-{{workflow.name}}"
  namespace: kubeflow
spec:
  annotations:
    project_name: Income Classifier Pipeline
    deployment_version: v1
  name: "seldon-deployment-{{workflow.name}}"
  oauth_key: oauth-key
  oauth_secret: oauth-secret
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: gcr.io/dev-joel/income_classifier:0.1
          imagePullPolicy: IfNotPresent
          name: incomeclassifier
          volumeMounts:
          - name: mypvc
            mountPath: /mnt
        terminationGracePeriodSeconds: 20
        volumes:
        - name: mypvc
          persistentVolumeClaim:
            claimName: "{{workflow.name}}-my-pvc"
    graph:
      children:
      - name: incomeclassifier
        endpoint:
          type: REST
        type: MODEL
      name: incomeclassifier
      endpoint:
        type: REST
      type: MODE

### Seldon Production pipeline contents
If we look at the file we'll be using to deploy our pipeline, we can see that it has the following key points:

1) Reusable components definitions as containerSpecs: income_classifier

2) DAG (directed acyclic graph) definition for REST pipeline: income_classifier

This graph in our production deployment looks as follows:

![](img/kubeflow-seldon-nlp-ml-pipelines-deploy.jpg)

### Generate the pipeline files to upload to Kubeflow
To generate the pipeline we just have to run the pipeline file, which will output the `tar.gz` file that will be uploaded.

In [58]:
%%bash
# Generating graph definition
python train_pipeline/income_pipeline.py
ls train_pipeline/

income_pipeline.py
income_pipeline.py.tar.gz
pipeline.yaml



### Run the pipeline

We now need to upload the resulting `nlp_pipeline.py.tar.gz` file generated.

This can be done through the "Upload PIpeline" button in the UI at http://localhost/_/pipeline-dashboard.

Once it's uploaded, we want to create and trigger a run! You should now be able to see how each step is executed:

![](img/running-pipeline.jpg)

### Inspecting the data created in the Persistent Volume
The pipeline saves the output of the pipeline together with the trained model in the persistent volume claim.

The persistent volume claim is the same name as the argo workflow:


In [59]:
!kubectl get workflow -n kubeflow

NAME        AGE
nlp-7zfn2   55m
nlp-8r495   28m
nlp-b6n8b   26m
nlp-c7bx4   23m
nlp-lcv7v   1h
nlp-phw95   51m
nlp-wdz7z   2m
nlp-wqkhb   16m


Our workflow is there! So we can actually access it by running

In [60]:
!kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}'

nlp-7zfn2

And we can use good old `sed` to insert this workflow name in our PVC-Access controler which we can use to inspect the contents of the volume:

In [61]:
!sed "s/PVC_NAME/"$(kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}')"-my-pvc/g" deploy_pipeline/pvc-access.yaml 

apiVersion: v1
kind: Pod
metadata:
  name: pvc-access-container
spec:
  containers:
  - name: pvc-access-container
    image: busybox
    command: ["/bin/sh", "-ec", "sleep 1000"]
    volumeMounts:
    - name: mypvc
      mountPath: /mnt
  volumes:
  - name: mypvc
    persistentVolumeClaim:
      claimName: nlp-7zfn2-my-pvc


We just need to apply this container with our kubectl command, and we can use it to inspect the mounted folder:

In [62]:
!sed "s/PVC_NAME/"$(kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}')"-my-pvc/g" deploy_pipeline/pvc-access.yaml | kubectl -n kubeflow apply -f -

pod/pvc-access-container created


In [64]:
!kubectl get pods -n kubeflow pvc-access-container

NAME                   READY   STATUS    RESTARTS   AGE
pvc-access-container   1/1     Running   0          6s


Now we can run an `ls` command to see what's inside:

In [65]:
!kubectl -n kubeflow exec -it pvc-access-container ls /mnt

In [66]:
!kubectl delete -f deploy_pipeline/pvc-access.yaml -n kubeflow

pod "pvc-access-container" deleted


## 5) Test Deployed ML REST Endpoints
Now that it's running we have a production ML text pipeline that we can Query using REST and GRPC


First we can check if our Seldon deployment is running with

In [67]:
!kubectl -n kubeflow get seldondeployment 

NAME                          AGE
seldon-deployment-nlp-wdz7z   1m


We will need the Seldon Pipeline Deployment name to reach the API, so we can get it using:

In [68]:
!kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}'

seldon-deployment-nlp-wdz7z

Now we can interact with our API in two ways: 

1) Using CURL or any client like PostMan

2) Using the Python SeldonClient

### Using CURL from the terminal
When using CURL, the only thing we need to provide is the data in JSON format, as well as the url, which is of the format:

```
http://<ENDPOINT>/seldon/kubeflow/<PIPELINE_NAME>/api/v0.1/predictions
```

In [69]:
%%bash
curl -X POST -H 'Content-Type: application/json' \
    -d "{'data': {'names': ['text'], 'ndarray': ['Hello world this is a test']}}" \
    http://127.0.0.1/seldon/kubeflow/$(kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}')/api/v0.1/predictions

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (7) Failed to connect to 127.0.0.1 port 80: Connection refused


CalledProcessError: Command 'b'curl -X POST -H \'Content-Type: application/json\' \\\n    -d "{\'data\': {\'names\': [\'text\'], \'ndarray\': [\'Hello world this is a test\']}}" \\\n    http://127.0.0.1/seldon/kubeflow/$(kubectl -n kubeflow get seldondeployment -o jsonpath=\'{.items[0].metadata.name}\')/api/v0.1/predictions\n'' returned non-zero exit status 7.

### Using the SeldonClient
We can also use the Python SeldonClient to interact with the pipeline we just deployed 

In [70]:
from seldon_core.seldon_client import SeldonClient
import numpy as np
import subprocess

host = "localhost"
port = "80" # Make sure you use the port above
batch = np.array(["Hello world this is a test"])
payload_type = "ndarray"
# Get the deployment name
deployment_name = subprocess.getoutput("kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}'")
transport="rest"
namespace="kubeflow"

sc = SeldonClient(
    gateway="ambassador", 
    ambassador_endpoint=host + ":" + port,
    namespace=namespace)

client_prediction = sc.predict(
    data=batch, 
    deployment_name=deployment_name,
    names=["text"],
    payload_type=payload_type,
    transport="rest")

print(client_prediction)

ImportError: Traceback (most recent call last):
  File "/Users/Seldon/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "/Users/Seldon/miniconda3/lib/python3.7/imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Seldon/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/Seldon/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/Seldon/miniconda3/lib

## 6) Visualise Seldon's Production ML Pipelines
We can visualise the performance using the SeldonAnalytics package, which we can deploy using:

In [71]:
!helm install seldon-core-analytics --repo https://storage.googleapis.com/seldon-charts --namespace kubeflow

NAME:   gangly-quail
LAST DEPLOYED: Wed Jul  3 17:48:56 2019
NAMESPACE: kubeflow
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     3s
grafana-import-dashboards  11    3s
prometheus-rules           0     3s
prometheus-server-conf     1     3s

==> v1/Job
NAME                            COMPLETIONS  DURATION  AGE
grafana-prom-import-dashboards  0/1          3s        3s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
alertmanager-deployment-79d4f8b64-7fn5k   0/1    ContainerCreating  0         3s
grafana-prom-deployment-687bc94bbb-f959w  0/1    ContainerCreating  0         2s
grafana-prom-import-dashboards-f5wvt      0/1    ContainerCreating  0         3s
prometheus-deployment-68b9445fb8-mjm9f    0/1    ContainerCreating  0         2s
prometheus-node-exporter-rhmlf            0/1    ContainerCreating  0         2s

==> v1/Secret
NAME                 TYPE    DATA  AGE
g

In my case, similar to what I did with Ambassador, I need to make sure the the service is a LoadBalancer instead of a NodePort

In [72]:
!kubectl patch svc grafana-prom --type='json' -p '[{"op":"replace","path":"/spec/type","value":"LoadBalancer"}]' -n kubeflow

service/grafana-prom patched


In [74]:
!kubectl get svc grafana-prom -n kubeflow

NAME           TYPE           CLUSTER-IP      EXTERNAL-IP       PORT(S)        AGE
grafana-prom   LoadBalancer   10.11.246.155   104.198.131.212   80:31680/TCP   48s


Now we can access it at the port provided, in my case it is http://localhost:32445/d/3swM2iGWz/prediction-analytics?refresh=5s&orgId=1

(initial username is admin and password is password, which will be requested to be changed on the first login)

Generate a bunch of requests and visualise:

In [ ]:
while True:
    client_prediction = sc.predict(
        data=batch, 
        deployment_name=deployment_name,
        names=["text"],
        payload_type=payload_type,
        transport="rest")

## You now have a full end-to-end training and production NLP pipeline 😎 
![](img/seldon-analytics.jpg)